In [11]:
# Install all the necessary python libraries if anaconda is not able to find them.

In [12]:
# Import all the necessary python libraries
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
from pprint import pprint

In [13]:
# Define CONSTANTS
FNB_IMAGE_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button/img'
FNB_OPENING_HOURS_BTN_DOWN_XPATH = '//*[@jsaction[starts-with(., "pane.openhours")]]' #and @role="button" and @tabindex="0" and @aria-expanded="false"]'
FNB_OPENING_HOURS_BTN_RIGHT_XPATH = '//*[@data-item-id="oh"]'
FNB_OPENING_HOURS_DETAILS_XPATH = '//*[@aria-label[contains(., "Hide open hours for the week")]]'
FNB_OPENING_HOURS_BACK_BTN_XPATH = '//*[@aria-label="Back"]'
FNB_OPENING_HOURS_CLOSED_XPATH = '//span[@class=" fCEvvc"]' #Temporarily Closed / #Permanently Closed
FNB_REVIEWS_TAB_XPATH = '//button[@role="tab" and @aria-label[starts-with(., "Reviews")]]'
FNB_SORT_REVIEWS_BTN_XPATH = '//button[@aria-label="Sort reviews"]'
FNB_SORT_BY_NEWEST_BTN_XPATH = '//div[@class="fxNQSd" and @data-index="1" and @role="menuitemradio"]'
FNB_REVIEW_LIST_XPATH = '//div[@class="jftiEf fontBodyMedium "]'
FNB_REVIEW_SCROLLABLE_CONTENT_XPATH = '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf "]'
FNB_REVIEW_COUNT_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span'#'//span[contains(@aria-label, "review")]'
FNB_REVIEW_SEE_MORE_XPATH = './/button[@aria-label="See more"]'
FNB_REVIEW_RATING_XPATH = './/span[@class="kvMYJc" and @role="img"]'
FNB_REVIEW_CONTENT_XPATH = './/span[@class="wiI7pd"]'
FNB_REVIEW_PHOTOS_XPATH = './/button[@class="Tya61d"]'
FNB_REVIEWER_XPATH = './/button[@class="al6Kxe"]'
FNB_REVIEWER_NAME_XPATH = './/div[@class="d4r55 "]'
FNB_REVIEWER_DETAILS_XPATH = './/div[@class="RfnDt "]'
FNB_CLOSED_XPATH = '//span[@class=" fCEvvc"]'

In [14]:
# Initialize Variables
review_list = []

In [15]:
file_path = 'csv_data/fnb_list_part_7_reviews.csv'
df_final = pd.read_csv(file_path, encoding='utf-8')
df_final.head(10)


,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Premise Name,Company Brand Name,Halal Certification Expiry Date,Halal Certification Body,Reviews,Opening Hours,Photo
0,Iron Chef House (non halal),Iron Chef House,"['Western Food', 'Cafe', 'Malay Food', 'Chines...",Seri Kembangan,Floor 2 · The Mines Shopping Mall,"E-2-G,E-3-G,E-4-G &E-5-G, Jalan PP 25, Putra W...",03-9544 4793,https://www.google.com/maps/place/Iron+Chef+Ho...,NaN,https://www.pizzahut.com.my/,https://www.foodpanda.my/restaurant/m7ax/pizza...,NaN,NaN,NaN,NaN,"[{'Rating': 5, 'Text': 'The food is good, the ...","Monday, 4 pm to 4 am; Tuesday, 4 pm to 4 am; W...",https://lh5.googleusercontent.com/p/AF1QipOxZO...
1,Big Family Restaurant,Big Family Restaurant,"['Western Food', 'Middle Eastern Food', 'Chine...",Subang Jaya,NaN,"2, Jalan SS 15/4b, Ss 15, 47500 Subang Jaya, S...",012-305 9350,https://www.google.com/maps/place/Big+Family+R...,NaN,https://www.facebook.com/%E5%A4%A7%E5%AE%B6%E7...,https://www.foodpanda.my/restaurant/l7vz/pork-...,NaN,NaN,NaN,NaN,"[{'Rating': 0, 'Text': None, 'Photos': [], 'Re...","Monday, 7 am to 3 am; Tuesday, 7 am to 3 am; W...",https://lh5.googleusercontent.com/p/AF1QipNOv_...
2,May Keng Cafe Sdn Bhd,May Keng Cafe Sdn Bhd,"['Western Food', 'Cafe', 'Chinese Food', 'Mala...",Seri Kembangan,NaN,"1-1, Jalan SP 2/4, Taman Serdang Perdana, 4330...",03-8943 8070,https://www.google.com/maps/place/May+Keng+Caf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'Rating': 4, 'Text': 'Tasty food. Good servi...","Monday, Closed; Tuesday, 10 am to 4 pm; Wednes...",https://lh5.googleusercontent.com/p/AF1QipOhGX...
3,Nasi Kandar Pelita,Nasi Kandar Pelita,"['Indian Food', 'Middle Eastern Food', 'Malay ...",Subang Jaya,NaN,"2, Jalan SS 15/4c, Ss 15, 47500 Subang Jaya, S...",03-5632 5532,https://www.google.com/maps/place/Nasi+Kandar+...,NaN,http://www.pelita.com.my/,https://www.foodpanda.my/restaurant/o1im/nasi-...,NASI KANDAR PELITA (SUBANG JAYA),NASI KANDAR PELITA SDN BHD,2020-07-15,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),"[{'Rating': 5, 'Text': None, 'Photos': [], 'Re...","Monday, Open 24 hours; Tuesday, Open 24 hours;...",https://lh5.googleusercontent.com/p/AF1QipNhAx...
4,All Want To Eat Restaurant,All Want To Eat Restaurant,"['Western Food', 'Chinese Food', 'Malay Food']",Seri Kembangan,NaN,"No. 192, Jalan SK 7/11, Kampung Baru Seri Kemb...",012-366 6863,https://www.google.com/maps/place/All+Want+To+...,NaN,NaN,https://www.foodpanda.my/restaurant/n1cx/all-w...,NaN,NaN,NaN,NaN,"[{'Rating': 1, 'Text': None, 'Photos': [], 'Re...","Monday, 11 am to 2:30 pm, 4:30 to 10:30 pm; Tu...",https://lh5.googleusercontent.com/p/AF1QipONW4...
5,Belly & The Chef Cafe (Cheras),Belly & The Chef Cafe,"['Western Food', 'Cafe', 'Dessert']",Cheras,NaN,"13, Lorong Durian, Taman Cheras, 56100 Kuala L...",012-527 3966,https://www.google.com/maps/place/Belly+%26+Th...,https://linktr.ee/bellyandthechef,https://linktr.ee/bellyandthechef,https://linktr.ee/bellyandthechef,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Canai 15 Cafè,Canai 15 Cafè,"['Western Food', 'Indian Food', 'Vegetarian Fo...",Subang Jaya,Sunway Pyramid,"99, Jalan SS 14/1, Ss 14, 47500 Subang Jaya, S...",03-5611 8426,https://www.google.com/maps/place/Canai+15+Caf...,https://food.grab.com/my/en/restaurant/canai-c...,https://canai-15-cafe-taste-of-india.business....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Chef Ammar Xpress Souk Cafe,Chef Ammar Xpress Souk Cafe,"['Western Food', 'Cafe', 'Middle Eastern Food']",Subang Jaya,NaN,"16, Jalan USJ 4/1, Usj 4, 47600 Subang Jaya, S...",03-8011 8073,https://www.google.com/maps/place/Chef+Ammar+X...,NaN,http://www.chefammar.com.my/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Day 9 Kitchen Cafe 明明,Day 9 Kitchen Cafe 明明,"['Western Food', 'Cafe', 'Japanese Food']",Seri Kembangan,NaN,"3-1, Jalan Equine 10B, Taman Equine, 43300 Ser...",016-616 0590,https://www.google.com/maps/place/Day+9+Kitche...,https://grab.onelink.me/2695613898?pid=inappsh...,https://www.facebook.com/m

In [16]:
def convert_opening_hours_to_dict(opening_hours):

    day_parts_1 = opening_hours.replace('\u202f', ' ')
    day_parts_2 = day_parts_1.split('.')[0]
    day_parts = day_parts_2.split(';')

    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    open_hours_dict = {
        part.split(',')[0].strip(): part.split(',', 1)[1].strip()
        for part in day_parts
    }

    sorted_open_hours_dict = {day: open_hours_dict[day] for day in days_order if day in open_hours_dict}

    return sorted_open_hours_dict


In [17]:
def parse_count_text(count):
    return int(count.replace(",", ""))

In [18]:
def parse_reviewer_details(reviewer_details):
    
    parts = [part.strip() for part in reviewer_details.split("·")]
    
    reviewer_type = None
    reviewer_reviews_count = 0
    reviewer_photos_count = None

    for part in parts:
        if 'Local Guide' in part:
            reviewer_type = 'Local Guide'
        elif 'review' in part:
            reviewer_reviews_count = parse_count_text(part.split()[0])
        elif 'photo' in part:
            reviewer_photos_count = parse_count_text(part.split()[0])
    
    return reviewer_type, reviewer_reviews_count, reviewer_photos_count

In [19]:
driver = webdriver.Safari()
driver.maximize_window()

In [21]:
for index, row in tqdm(df_final.iterrows(), desc="Processing rows"):
    url = row['Google Map Url']
    driver.get(url)
    fnb_image_element = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, FNB_IMAGE_XPATH))
    )
    fnb_image_url = fnb_image_element.get_attribute('src')
    #tqdm.write(f'{index+1}. {fnb_image_url}')
    time.sleep(1)
    total_reviews_count = 0
    current_reviews_count = 0
    review_list = []
    opening_hours_dict = None

    try:
        review_count_element = driver.find_element(By.XPATH, FNB_REVIEW_COUNT_XPATH)
        if review_count_element:
            total_reviews_count = parse_count_text(review_count_element.text.split("(")[1].split(")")[0])
            
    except:
        total_reviews_count = 0

    try:
        # CASE 1: OPENING HOURS BUTTON IS POINTING DOWN
        opening_hours_btn_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_DOWN_XPATH)
        time.sleep(1)
        try:
            opening_hours_btn_elem.click()
        except:
            opening_hours_btn_elem.send_keys(Keys.ENTER)
        time.sleep(1)
        opening_hours_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
        opening_hours = opening_hours_elem.get_attribute('aria-label')
        opening_hours_dict = convert_opening_hours_to_dict(opening_hours)

    except NoSuchElementException:

        # CASE 2: OPENING HOURS BUTTON IS POINTING RIGHT
        try:
            opening_hours_btn_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_RIGHT_XPATH)
            time.sleep(1)
            try:
                opening_hours_btn_elem.click()
            except:
                opening_hours_btn_elem.send_keys(Keys.ENTER)
            time.sleep(1)
            opening_hours_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
            opening_hours = opening_hours_elem.get_attribute('aria-label')
            opening_hours_dict = convert_opening_hours_to_dict(opening_hours)
            time.sleep(0.5)
            driver.find_element(By.XPATH, FNB_OPENING_HOURS_BACK_BTN_XPATH).click()
            time.sleep(1)

        except NoSuchElementException:

            # CASE 3: FNB IS TEMPORARILY CLOSED OR PERMANENTLY CLOSED
            try:
                opening_hours_dict = driver.find_element(By.XPATH, FNB_OPENING_HOURS_CLOSED_XPATH).text

            except NoSuchElementException:
                
                # CASE 4: NO OPENING HOURS STATED
                opening_hours_dict = None
            time.sleep(1)
        pass

    #tqdm.write(f'{index+1}. {opening_hours_dict}')

    if total_reviews_count != 0:
        driver.find_element(By.XPATH, FNB_REVIEWS_TAB_XPATH).click()
        time.sleep(1)

        driver.find_element(By.XPATH, FNB_SORT_REVIEWS_BTN_XPATH).click()
        time.sleep(1)
        
        driver.find_element(By.XPATH, FNB_SORT_BY_NEWEST_BTN_XPATH).click()
        time.sleep(1)

        while True:

            scrollable_review_content_element = driver.find_element(By.XPATH, FNB_REVIEW_SCROLLABLE_CONTENT_XPATH)
            review_list_element = driver.find_elements(By.XPATH, FNB_REVIEW_LIST_XPATH)
            time.sleep(0.5)
            current_reviews_count = len(review_list_element)
            driver.execute_script(
                'arguments[0].scrollTop = arguments[0].scrollHeight', 
                scrollable_review_content_element
            )

            if current_reviews_count >= 100:
                break

            elif current_reviews_count == total_reviews_count:
                break
        
        for review in review_list_element:
        #review = review_list_element[14]
            review_rating = 0
            review_text = None
            review_photos = []
            reviewer_name = ''
            reviewer_url = ''
            reviewer_type = None
            reviewer_reviews_count = 0
            reviewer_photos_count = None
            
            reviewer_element = review.find_element(By.XPATH, FNB_REVIEWER_XPATH)

            retry_count = 0
            while retry_count < 2:
                try:
                    reviewer_url = reviewer_element.get_attribute("data-href")
                    time.sleep(0.5)

                    reviewer_name = reviewer_element.find_element(By.XPATH, FNB_REVIEWER_NAME_XPATH).text
                    time.sleep(0.5)

                    reviewer_details = reviewer_element.find_element(By.XPATH, FNB_REVIEWER_DETAILS_XPATH).text
                    time.sleep(0.5)

                    reviewer_type, reviewer_reviews_count, reviewer_photos_count = parse_reviewer_details(reviewer_details)
                    time.sleep(0.5)

                    review_rating_element = review.find_element(By.XPATH, FNB_REVIEW_RATING_XPATH)
                    review_rating = int(review_rating_element.get_attribute('aria-label').split()[0])
                    time.sleep(0.5)

                    break
                except:
                    retry_count = retry_count + 1
            try:
                see_more_btn = review.find_element(By.XPATH, FNB_REVIEW_SEE_MORE_XPATH)
                if see_more_btn:
                    see_more_btn.click()
                    time.sleep(0.5)
            except:
                see_more_btn = None
            
            try:
                review_content_element = review.find_element(By.XPATH, FNB_REVIEW_CONTENT_XPATH)
                if review_content_element:
                    review_text = review_content_element.text
                    time.sleep(0.5)
            except:
                review_text = None
            
            try:
                review_photos_element = review.find_elements(By.XPATH, FNB_REVIEW_PHOTOS_XPATH)
                time.sleep(0.5)
                if review_photos_element:
                    for photos in review_photos_element:
                        photo_style_attribute = photos.get_attribute('style')
                        url_match = re.search(r'url\("([^"]+)"\)', photo_style_attribute)
                        photo_url = url_match.group(1)
                        review_photos.append(photo_url)
            except:
                review_photos = None
        
            review = {
                "Rating": review_rating,
                "Text": review_text,
                "Photos": review_photos,
                "Reviewer": {
                    "Name": reviewer_name,
                    "Reviews Count": reviewer_reviews_count,
                    "Photos Count": reviewer_photos_count,
                    "Type": reviewer_type,
                    "Url": reviewer_url
                }
            }
            review_list.append(review)
        df_final.at[index, 'Reviews'] = review_list
    df_final.at[index, 'Opening Hours'] = opening_hours_dict
    df_final.at[index, 'Photo'] = fnb_image_url
    df_final.to_csv('csv_data/fnb_list_part_7_reviews.csv', index=False)

Processing rows: 1it [00:06,  6.07s/it]

{'Monday': '4 pm to 4 am', 'Tuesday': '4 pm to 4 am', 'Wednesday': '4 pm to 4 am', 'Thursday': '4 pm to 4 am', 'Friday': '4 pm to 4 am', 'Saturday': '4 pm to 4 am', 'Sunday': '4 pm to 4 am'}


Processing rows: 2it [00:10,  5.23s/it]

{'Monday': '7 am to 3 am', 'Tuesday': '7 am to 3 am', 'Wednesday': '7 am to 3 am', 'Thursday': '7 am to 3 am', 'Friday': '7 am to 3 am', 'Saturday': '7 am to 3 am', 'Sunday': '7 am to 3 am'}


Processing rows: 3it [00:14,  4.76s/it]

...

{'Monday': 'Open 24 hours', 'Tuesday': 'Open 24 hours', 'Wednesday': 'Open 24 hours', 'Thursday': 'Open 24 hours', 'Friday': 'Open 24 hours', 'Saturday': 'Open 24 hours', 'Sunday': 'Open 24 hours'}


Processing rows: 5it [00:23,  4.57s/it]

{'Monday': '11 am to 2:30 pm, 4:30 to 10:30 pm', 'Tuesday': '11 am to 2:30 pm, 4:30 to 10:30 pm', 'Wednesday': '11 am to 2:30 pm, 4:30 to 10:30 pm', 'Thursday': 'Closed', 'Friday': '11 am to 2:30 pm, 4:30 to 10:30 pm', 'Saturday': '11 am to 2:30 pm, 4:30 to 10:30 pm', 'Sunday': '11 am to 2:30 pm, 4:30 to 10:30 pm'}


Processing rows: 5it [00:26,  5.32s/it]


KeyboardInterrupt: 

In [ ]:
#file_path = 'csv_data/fnb_list_part_7_reviews.csv'
#df_final = pd.read_csv(file_path, encoding='utf-8')
df_final.head(10)

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Premise Name,Company Brand Name,Halal Certification Expiry Date,Halal Certification Body,Reviews,Opening Hours,Photo
0,Iron Chef House (non halal),Iron Chef House,"['Western Food', 'Cafe', 'Malay Food', 'Chines...",Seri Kembangan,Floor 2 · The Mines Shopping Mall,"E-2-G,E-3-G,E-4-G &E-5-G, Jalan PP 25, Putra W...",03-9544 4793,https://www.google.com/maps/place/Iron+Chef+Ho...,NaN,https://www.pizzahut.com.my/,https://www.foodpanda.my/restaurant/m7ax/pizza...,NaN,NaN,NaN,NaN,"[{'Rating': 5, 'Text': 'The food is good, the ...",NaN,NaN
1,Big Family Restaurant,Big Family Restaurant,"['Western Food', 'Middle Eastern Food', 'Chine...",Subang Jaya,NaN,"2, Jalan SS 15/4b, Ss 15, 47500 Subang Jaya, S...",012-305 9350,https://www.google.com/maps/place/Big+Family+R...,NaN,https://www.facebook.com/%E5%A4%A7%E5%AE%B6%E7...,https://www.foodpanda.my/restaurant/l7vz/pork-...,NaN,NaN,NaN,NaN,"[{'Rating': 0, 'Text': None, 'Photos': [], 'Re...",NaN,NaN
2,May Keng Cafe Sdn Bhd,May Keng Cafe Sdn Bhd,"['Western Food', 'Cafe', 'Chinese Food', 'Mala...",Seri Kembangan,NaN,"1-1, Jalan SP 2/4, Taman Serdang Perdana, 4330...",03-8943 8070,https://www.google.com/maps/place/May+Keng+Caf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'Rating': 4, 'Text': 'Tasty food. Good servi...",NaN,NaN
3,Nasi Kandar Pelita,Nasi Kandar Pelita,"['Indian Food', 'Middle Eastern Food', 'Malay ...",Subang Jaya,NaN,"2, Jalan SS 15/4c, Ss 15, 47500 Subang Jaya, S...",03-5632 5532,https://www.google.com/maps/place/Nasi+Kandar+...,NaN,http://www.pelita.com.my/,https://www.foodpanda.my/restaurant/o1im/nasi-...,NASI KANDAR PELITA (SUBANG JAYA),NASI KANDAR PELITA SDN BHD,2020-07-15,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),"[{'Rating': 5, 'Text': None, 'Photos': [], 'Re...",NaN,NaN
4,All Want To Eat Restaurant,All Want To Eat Restaurant,"['Western Food', 'Chinese Food', 'Malay Food']",Seri Kembangan,NaN,"No. 192, Jalan SK 7/11, Kampung Baru Seri Kemb...",012-366 6863,https://www.google.com/maps/place/All+Want+To+...,NaN,NaN,https://www.foodpanda.my/restaurant/n1cx/all-w...,NaN,NaN,NaN,NaN,"[{'Rating': 1, 'Text': None, 'Photos': [], 'Re...",NaN,NaN
5,Belly & The Chef Cafe (Cheras),Belly & The Chef Cafe,"['Western Food', 'Cafe', 'Dessert']",Cheras,NaN,"13, Lorong Durian, Taman Cheras, 56100 Kuala L...",012-527 3966,https://www.google.com/maps/place/Belly+%26+Th...,https://linktr.ee/bellyandthechef,https://linktr.ee/bellyandthechef,https://linktr.ee/bellyandthechef,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Canai 15 Cafè,Canai 15 Cafè,"['Western Food', 'Indian Food', 'Vegetarian Fo...",Subang Jaya,Sunway Pyramid,"99, Jalan SS 14/1, Ss 14, 47500 Subang Jaya, S...",03-5611 8426,https://www.google.com/maps/place/Canai+15+Caf...,https://food.grab.com/my/en/restaurant/canai-c...,https://canai-15-cafe-taste-of-india.business....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Chef Ammar Xpress Souk Cafe,Chef Ammar Xpress Souk Cafe,"['Western Food', 'Cafe', 'Middle Eastern Food']",Subang Jaya,NaN,"16, Jalan USJ 4/1, Usj 4, 47600 Subang Jaya, S...",03-8011 8073,https://www.google.com/maps/place/Chef+Ammar+X...,NaN,http://www.chefammar.com.my/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Day 9 Kitchen Cafe 明明,Day 9 Kitchen Cafe 明明,"['Western Food', 'Cafe', 'Japanese Food']",Seri Kembangan,NaN,"3-1, Jalan Equine 10B, Taman Equine, 43300 Ser...",016-616 0590,https://www.google.com/maps/place/Day+9+Kitche...,https://grab.onelink.me/2695613898?pid=inappsh...,https://www.facebook.com/mingday9kitchen/,https://grab.onelink.me/2695613898?pid=inappsh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Good Friends Restaurant & Cafe,Good Friends Restaurant & Cafe,"['Western Food', 'Cafe', 'Middle Eastern Food']",Subang Jaya,NaN,"55, Jalan SS 15/8a, Ss 15, 47500 Subang Jaya, ...",03-5613 1415,https://www.google.com/maps/place/Good+Friends...,http://www.goodfriendscafe.com.my/,http://www.goodfriendscafe.com.my/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
